# Transformation Interpolations:  DES DR2 <--> Stetson UBVRI

_Douglas L. Tucker_

_2021.03.17_

## Initial Setup

In [1]:
import numpy as np
import pandas as pd

import os
import sys
import glob
import math
import datetime

from collections import OrderedDict as odict

from astropy.io import fits
from astropy.table import Table

import fitsio

from scipy import interpolate
from scipy.optimize import leastsq

import healpy as hp
import healpixTools

import extinction

import paramFile

import plotly
from plotly.offline import download_plotlyjs, plot, iplot
import plotly.graph_objs as go

import matplotlib.pyplot as plt

%matplotlib inline

## Input Parameters

In [2]:
# Stetson-->DES (toDES=True) or DES-->Stetson (toDES=False)...
#toDES = True
toDES = False


if toDES:
    # List of DES bands on which to run the transform fit...
    bandList = ['g_des', 'r_des', 'i_des', 'z_des', 'Y_des']
    # If Stetson-->DES, then the mag and color to fit against are Stetson...
    # Dictionary of corresponding bands from the other survey...
    matchBand_dict = {'g_des':'V', 'r_des':'R', 'i_des':'I', 'z_des':'I', 'Y_des':'I'}
    # Color to fit against...
    color_name_1_dict = {'g_des':'BV', 'r_des':'RI', 'i_des':'RI', 'z_des':'RI', 'Y_des':'RI'}
    # Name of color_name_1 as the label in the QA plots...
    colorLabel_1_dict = {'g_des':'$(B-V)$', 'r_des':'$(R-I)$', 'i_des':'$(R-I)$', 'z_des':'$(R-I)$', 'Y_des':'$(R-I)$'}

else:    
    # List of Stetson bands on which to run the transform fit...
    bandList = ['V']
    # If DES-->Stetson, then the mag and color to fit against are DES...
    # Dictionary of corresponding bands from the other survey...
    matchBand_dict = {'B':'g_des', 'V':'g_des', 'R':'r_des', 'I':'i_des'}
    # Color to fit against...
    color_name_1_dict = {'B':'gi_des', 'V':'gi_des', 'R':'gi_des', 'I':'gi_des'}
    # Name of color_name_1 as the label in the QA plots...
    colorLabel_1_dict = {'B':'$(g-i)_{des}$', 'V':'$(g-i)_{des}$', 'R':'$(g-i)_{des}$', 'I':'$(g-i)_{des}$'}


# Size of the binsize in color
color_binsize = 0.10

# Identify directory containing the DES/Stetson match file...
matchDir = './Stetson_UBVRI_data'

# Name of the DES/Stetson match file in matchDir
matchFile = 'match.Stetson_UBVRI.Y6A2_COADD.fits'


# Base name of fit results output file...
if toDES:
    resultsFileBaseName = 'transInterp.stetson_to_des.tingli'
else:
    resultsFileBaseName = 'transInterp.des_to_stetson.tingli'

# Base name of QA plot output files...
if toDES:
    qaFileBaseName = 'qaPlot.stetson_to_des.interp.tingli'
else:
    qaFileBaseName = 'qaPlot.des_to_stetson.interp.tingli'

# Verbosity level (0, 1, 2, 3, ...)
verbose = 2

## Define Some Modules

In [3]:
# From Zeljko Ivezic:
# Robust estimate of standard deviation: 0.741*(q75-q25)
#  sigmaG = 0.741*(np.percentile(yAux,75)-np.percentile(yAux,25))
# Uncertainty of the median: sqrt(pi/2)*st.dev/sqrt(N)
#  sigGbin[i] = np.sqrt(np.pi/2)*sigmaG/np.sqrt(nPts[i])

def robustStdDev(data):
    try:
        sd = 0.741*(np.percentile(data,75)-np.percentile(data,25))
    except:
        sd = -9999.
    return sd

def uncMedian(data, npts):
    sd = robustStdDev(data)
    try:
        uncMedian = np.sqrt(np.pi/2)*sd/np.sqrt(npts)
    except:
        uncMedian = -9999.
    return uncMedian

In [4]:
##################################

def interp1ColorQAPlots1(df, df1, color_name_1, color1_min, color1_max, 
                         title, dmagLabel, outputFileName, verbose):

        
    # Create linear interpolation of the median dmag vs. color bin calculated above...
    response = interpolate.interp1d(df1.index.values.astype(float), df1.bin_median.values, \
                                bounds_error=False, fill_value=0., kind='linear')    

    # Calculate and apply the offset for the individual stars in df, 
    #  based on the linear interpolation calculated above...
    df['offset'] = response(df[color_name_1].values)    
    df['res'] = df['dmag'] - df['offset']    
    if verbose>1:
        print 
        print df[[color_name_1, 'offset', 'res']].head()
        print 

    # Calculate the 'robust' RMS of the residuals....
    rms = robustStdDev(df['res'].values)


    # Identify range to plot in the dmag (Y-axis) direction...
    dmag_desc = df['dmag'].describe(percentiles=[0.001, 0.999])
    if verbose>1:
        print dmag_desc
    dmag_min = math.floor(10*dmag_desc['0.1%'])/10.
    dmag_max = math.ceil(10*dmag_desc['99.9%'])/10.

    dmag_mask = (df['dmag'] >= dmag_min) & (df['dmag'] <= dmag_max)
    df_dmag_mask = df[dmag_mask]
    

    # Identify range to plot in the residuals (Y-axis) direction...
    res_desc = df['res'].describe(percentiles=[0.01, 0.99])
    if verbose>1:
        print 
        print res_desc
        print 
    #res_min = math.floor(10*dmag_desc['1%'])/10.
    #res_max = math.ceil(10*dmag_desc['99%'])/10.
    # What the heck; let's just set this to -0.10 mag --> +0.10 mag...
    res_min = -0.10
    res_max = +0.10


    # For 2d binning below, we mask df for just those stars within 
    #  the res_min, res_max range...
    res_mask = (df['res'] >= res_min) & (df['res'] <= res_max)
    df_res_mask = df[res_mask]
    
    
    # Prepare QA plots...
    #fig = plt.figure(figsize=(40,20))
    fig = plt.figure(figsize=(20,10))
    fig.subplots_adjust(hspace=0.3)
    
    # Modify matplotlib font...
    #font = {'family' : 'normal',
    #        'weight' : 'bold',
    #       'size'   : 22}
    #matplotlib.rc('font', **font)    
    plt.rcParams.update({'font.size': 24})

    
    ## Include a supertitle to the matplotlob figure...
    #fig.suptitle('This is the superTitle')

    
    # Plot 1:  Descriptive text...
    plt.subplot(221)
    plot1Text = """%s \n\n\n\n%s vs. %s\n\n[rms: %.3f] \n\n[%.1f < %s < %.1f]""" % \
                    (title, dmagLabel, colorLabel_1, rms, color1_min, colorLabel_1, color1_max)
    plt.text(0.1,0.00,plot1Text)
    plt.axis('off')

    
    # Plot 2:  2D hexbin histogram of dmag vs. color1...
    plt.subplot(222)
    hb=plt.hexbin(df_dmag_mask[color_name_1].values, df_dmag_mask['dmag'].values, \
                      gridsize=100, bins='log', cmap='inferno')
    ax=plt.axis([color1_min, color1_max, dmag_min, dmag_max])
    plt.xlabel(colorLabel_1)
    plt.ylabel(dmagLabel)
    cb = fig.colorbar(hb)
    #cb.set_label('Number')
    cb.set_label('log10(N)')
    plt.grid(color='white')
    plt.grid(True)
    # Overplot the binned values calculated above:
    plt.errorbar(df1.index.values.astype(float), df1.bin_median.values, yerr=df1.bin_unc.values, fmt='o')

    
    # Plot 3:  1d histogram of residuals...
    plt.subplot(223) 
    plt.hist(df_res_mask.loc[:,'res'],bins=100)
    plt.xlabel('residuals [mag]')
    plt.ylabel('Number')
    plt.xlim([res_min, res_max])
    plt.grid(True)
    plt.grid(color='black')

    
    # Plot 4:  2d hexbin histogram of residuals vs. color1...
    plt.subplot(224) 
    hb=plt.hexbin(df_res_mask.loc[:, color_name_1].values, df_res_mask.loc[:,'res'].values, \
                  gridsize=100, bins='log', cmap='inferno')
    ax=plt.axis([color1_min, color1_max, res_min, res_max])
    plt.xlabel(colorLabel_1)
    plt.ylabel('Residuals')
    cb = fig.colorbar(hb)
    #cb.set_label('Number')
    cb.set_label('log10(N)')
    plt.grid(color='white')
    plt.grid(True)
    
    
    # Plot...
    plt.tight_layout()
    

    # Save QA plots to file...
    plt.savefig(outputFileName)    
    

    return 0


##################################

## Read in Matched Catalog

In [5]:
matchFile = os.path.join(matchDir,matchFile)
print matchFile

./Stetson_UBVRI_data/match.Stetson_UBVRI.Y6A2_COADD.fits


In [6]:
# Check to make sure matchFile exists...
if os.path.isfile(matchFile)==False:
    print """ERROR:  matchFile %s does not exist...""" % (matchFile)
if verbose > 0:
    print 'matchFile: ', matchFile


matchFile:  ./Stetson_UBVRI_data/match.Stetson_UBVRI.Y6A2_COADD.fits


In [7]:
tab = Table.read(matchFile, format='fits')
tab

<Table length=19605>
  name_pho    mag_U  sigma_U ... NEPOCHS_Y  MOLY_NUMBER_Y        Separation     
                             ...                                   arcsec       
   str12     float32 float32 ...   int32        int64             float64       
------------ ------- ------- ... --------- ---------------- --------------------
AB_Lep-S2     99.999  9.9999 ...         6 2852916500002808 0.046234798073801306
AB_Lep-S6     99.999  9.9999 ...         5 2852916500002807 0.040961115253143186
AB_Lep-S8     99.999  9.9999 ...         3 2852916500002807  0.07396702938395526
AM1-S1        99.999  9.9999 ...         8 2840886500001558  0.08273517899925154
AM1-S2        99.999  9.9999 ...         6 2840886500001600  0.07188478347222677
AM1-S3        99.999  9.9999 ...         8 2840886500001558  0.06476928058141801
AM1-S4        99.999  9.9999 ...         7 2840886500001599  0.07618602448207847
AM1-S5        99.999  9.9999 ...         6 2840886500001601  0.05664128136552577
AM1-S6        99.999  9.9999 ...         7 2840886500001602  0.06736386788408089
         ...     ...     ... ...       ...              ...                  ...
T_Phe-S398      16.8  0.0089 ...         6 2763956500004043  0.07815678173144605
T_Phe-S399    99.999  9.9999 ...         9 2763956500002712  0.06773028721689463
T_Phe-S400    99.999  9.9999 ...         7 2763956500001359  0.04317499961592413
T_Phe-S401    99.999  9.9999 ...         6 2763956500001286   0.8943901060352917
T_Phe-S403    99.999  9.9999 ...         6 2763956500004982  0.09372616698819941
T_Phe-S404    17.778  0.0196 ...         7 2763956500002565  0.19406272325567184
T_Phe-S405    16.962  0.0187 ...         8 2763956500001400  0.06438158984615562
T_Phe-S406    99.999  9.9999 ...         6 2763956500003936  0.07082316270256821
T_Phe-S407    16.519  0.0143 ...         6 2763956500002392  0.09210681320097856
T_Phe-S408     17.73  0.0152 ...         6 2763956500003939  0.05520171897602583

In [8]:
df = tab.to_pandas()
df

name_pho      mag_U  sigma_U  N_U  n_U      mag_B  sigma_B  N_B  \
0      AB_Lep-S2     99.999001   9.9999    0    0  99.999001   9.9999    0   
1      AB_Lep-S6     99.999001   9.9999    0    0  16.681000   0.0152    9   
2      AB_Lep-S8     99.999001   9.9999    0    0  99.999001   9.9999    0   
3      AM1-S1        99.999001   9.9999    0    0  18.752001   0.0039   19   
4      AM1-S2        99.999001   9.9999    0    0  19.799999   0.0083   19   
5      AM1-S3        99.999001   9.9999    0    0  16.813000   0.0041   19   
6      AM1-S4        99.999001   9.9999    0    0  17.575001   0.0097   19   
7      AM1-S5        99.999001   9.9999    0    0  19.789000   0.0110   19   
8      AM1-S6        99.999001   9.9999    0    0  18.181000   0.0042   19   
9      AM1-S7        99.999001   9.9999    0    0  18.319000   0.0033   19   
10     AM1-S8        99.999001   9.9999    0    0  18.936001   0.0036   19   
11     AM1-S10       99.999001   9.9999    0    0  99.999001   9.9999    0   
12     AM1-S11       99.999001   9.9999    0    0  19.344000   0.0090   19   
13     AM1-S13       99.999001   9.9999    0    0  20.350000   0.0050   19   
14     AM1-S14       99.999001   9.9999    0    0  19.430000   0.0035   19   
15     AM1-S15       99.999001   9.9999    0    0  20.292999   0.0069   19   
16     AM1-S16       99.999001   9.9999    0    0  18.402000   0.0051   19   
17     AM1-S17       99.999001   9.9999    0    0  18.212000   0.0067   19   
18     AM1-S19       99.999001   9.9999    0    0  16.681000   0.0030   25   
19     AM1-S20       99.999001   9.9999    0    0  16.847000   0.0079   23   
20     AM1-S21       99.999001   9.9999    0    0  16.452000   0.0036   25   
21     AM1-S22       99.999001   9.9999    0    0  17.952999   0.0040   19   
22     AM1-S23       99.999001   9.9999    0    0  20.014999   0.0034   23   
23     AM1-S24       99.999001   9.9999    0    0  20.747000   0.0041   18   
24     AM1-S26       99.999001   9.9999    0    0  18.607000   0.0032   24   
25     AM1-S27       99.999001   9.9999    0    0  17.947001   0.0029   24   
26     AM1-S28       99.999001   9.9999    0    0  19.233000   0.0071   23   
27     AM1-S29       99.999001   9.9999    0    0  19.511000   0.0053   25   
28     AM1-S30       99.999001   9.9999    0    0  19.235001   0.0049   23   
29     AM1-S32       99.999001   9.9999    0    0  20.948000   0.0054   15   
...             ...        ...      ...  ...  ...        ...      ...  ...   
19575  T_Phe-S375    16.132000   0.0165   12   13  16.131001   0.0033   55   
19576  T_Phe-S376    99.999001   9.9999    0    0  19.792000   0.0074   36   
19577  T_Phe-S377    99.999001   9.9999    0    0  17.490000   0.0059   37   
19578  T_Phe-S378    99.999001   9.9999    0    0  20.695999   0.0098   24   
19579  T_Phe-S379    17.934000   0.0152    9    9  17.584999   0.0058   28   
19580  T_Phe-S381    17.742001   0.0074   34   41  17.848000   0.0021  140   
19581  T_Phe-S382    18.841999   0.0091   30   35  18.844999   0.0032  133   
19582  T_Phe-S383    99.999001   9.9999    0    0  18.875999   0.0052   49   
19583  T_Phe-S384    99.999001   9.9999    0    0  17.332001   0.0040   54   
19584  T_Phe-S385    99.999001   9.9999    0    0  18.889999   0.0060   25   
19585  T_Phe-S388    99.999001   9.9999    0    0  20.084000   0.0059   61   
19586  T_Phe-S389    99.999001   9.9999    0    0  20.511000   0.0129   30   
19587  T_Phe-S390    16.118999   0.0159   14   15  16.222000   0.0021   57   
19588  T_Phe-S391    99.999001   9.9999    0    0  19.362000   0.0067   43   
19589  T_Phe-S392    16.715000   0.0187    9    9  16.365999   0.0046   40   
19590  T_Phe-S393    99.999001   9.9999    0    0  20.742001   0.0125   32   
19591  T_Phe-S394    18.871000   0.0137   27   30  17.910999   0.0026   83   
19592  T_Phe-S395    99.999001   9.9999    0    0  18.993000   0.0065   73   
19593  T_Phe-S396    99.999001   9.9999    0    0  18.827000   0.0101   22   
19594  T_Phe-S397    99.999001   

In [9]:
# Rename columns...
df.rename(columns={'RA':'RA_des',
                   'DEC':'DEC_des',
                   'raDeg':'RA_stetson',
                   'decDeg':'DEC_stetson',
                   'WAVG_MAG_PSF_G':'g_des',
                   'WAVG_MAG_PSF_R':'r_des',
                   'WAVG_MAG_PSF_I':'i_des',
                   'WAVG_MAG_PSF_Z':'z_des',
                   'WAVG_MAG_PSF_Y':'Y_des',
                   'WAVG_MAGERR_PSF_G':'g_err_des',
                   'WAVG_MAGERR_PSF_R':'r_err_des',
                   'WAVG_MAGERR_PSF_I':'i_err_des',
                   'WAVG_MAGERR_PSF_Z':'z_err_des',
                   'WAVG_MAGERR_PSF_Y':'Y_err_des',
                   'B':'B_LAT',
                   'G':'G_LONG',
                   'mag_U':'U',
                   'mag_B':'B',
                   'mag_V':'V',
                   'mag_R':'R',
                   'mag_I':'I',
                   'sigma_U':'U_err',
                   'sigma_B':'B_err',
                   'sigma_V':'V_err',
                   'sigma_R':'R_err',
                   'sigma_I':'I_err',
                  },inplace=True)

df.head(5)

name_pho          U   U_err  N_U  n_U          B   B_err  N_B  n_B  \
0  AB_Lep-S2     99.999001  9.9999    0    0  99.999001  9.9999    0    0   
1  AB_Lep-S6     99.999001  9.9999    0    0  16.681000  0.0152    9    9   
2  AB_Lep-S8     99.999001  9.9999    0    0  99.999001  9.9999    0    0   
3  AM1-S1        99.999001  9.9999    0    0  18.752001  0.0039   19   27   
4  AM1-S2        99.999001  9.9999    0    0  19.799999  0.0083   19   26   

           V     ...      SPREAD_MODEL_Y  CLASS_STAR_Y  NITER_MODEL_Y  \
0  16.277000     ...            0.000080      0.824420             31   
1  16.160000     ...            0.000713      0.910380             36   
2  16.108999     ...            0.001047      0.975248             31   
3  17.983999     ...            0.000182      0.956209             27   
4  18.365000     ...           -0.001730      0.980777             24   

   WAVG_SPREADERR_MODEL_Y  WAVG_SPREAD_MODEL_Y  FLAGS_Y  IMAFLAGS_ISO_Y  \
0                0.000044            -0.000286        2               0   
1                0.000045             0.000031        0               0   
2                0.000044             0.000407        2               0   
3                0.000106            -0.000195        2               0   
4                0.000063            -0.000237        0               0   

   NEPOCHS_Y     MOLY_NUMBER_Y  Separation  
0          6  2852916500002808    0.046235  
1          5  2852916500002807    0.040961  
2          3  2852916500002807    0.073967  
3          8  2840886500001558    0.082735  
4          6  2840886500001600    0.071885  

[5 rows x 218 columns]

## Add Columns to Matched Catalog Data Frame

In [10]:
# Add color columns...
df.loc[:,'gr_des'] = df.loc[:,'g_des'] - df.loc[:,'r_des']
df.loc[:,'ri_des'] = df.loc[:,'r_des'] - df.loc[:,'i_des']
df.loc[:,'iz_des'] = df.loc[:,'i_des'] - df.loc[:,'z_des']
df.loc[:,'zY_des'] = df.loc[:,'z_des'] - df.loc[:,'Y_des']
df.loc[:,'gi_des'] = df.loc[:,'g_des'] - df.loc[:,'i_des']

df.loc[:,'UB'] = df.loc[:,'U'] - df.loc[:,'V']
df.loc[:,'BV'] = df.loc[:,'B'] - df.loc[:,'V']
df.loc[:,'VR'] = df.loc[:,'V'] - df.loc[:,'R']
df.loc[:,'RI'] = df.loc[:,'R'] - df.loc[:,'I']

In [11]:
# Insert dmag column...
df.loc[:,'dmag'] = -9999.

## Create Initial Mask

In [12]:
mask1 = df["N_B"] >= 5
mask2 = df["N_V"] >= 5
mask3 = df["N_R"] >= 5
mask4 = df["N_I"] >= 5
mask5 = df["B_err"] <= 0.02
mask6 = df["V_err"] <= 0.02
mask7 = df["R_err"] <= 0.02
mask8 = df["I_err"] <= 0.02
mask_stetson = mask1 & mask2 & mask3 & mask4 & mask5 & mask6 & mask7 & mask8

mask1 = df["g_des"] > 0.
mask2 = df["r_des"] > 0.
mask3 = df["i_des"] > 0.
mask4 = df["z_des"] > 0.
mask5 = df["Y_des"] > 0.
mask6 = df["g_err_des"] <= 0.02
mask7 = df["r_err_des"] <= 0.02
mask8 = df["i_err_des"] <= 0.02
mask9 = df["z_err_des"] <= 0.02
mask10 = df["Y_err_des"] <= 0.02
mask_des = mask1 & mask2 & mask3 & mask4 & mask5 & mask6 & mask7 & mask8 & mask9 & mask10

mask = mask_stetson & mask_des

## Make Backup Copies of Initial Mask and Original Data Frame

In [13]:
# Make a backup copy of original df...
df_orig = df.copy()

# Make a backup copy of original mask...
mask_orig = mask.copy()

In [14]:
#df.describe()
band = 'B'
magName = band
magName_match = matchBand_dict[band]
color_name_1 = color_name_1_dict[band]
colorLabel_1 = colorLabel_1_dict[band]
print magName_match
print color_name_1
print colorLabel_1

g_des
gi_des
$(g-i)_{des}$


## Run Fit in Each Filter Band

In [15]:
for band in bandList:
    
    print 
    print 
    print 
    print "# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # "
    print band
    print "# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # "
    print 
    
    magName = band
    magName_match = matchBand_dict[band]
    color_name_1 = color_name_1_dict[band]
    colorLabel_1 = colorLabel_1_dict[band]

    
    # Grab the original version of df from the backup copy...
    df = df_orig.copy()

    # Grab the original version of mask from the backup copy...
    mask = mask_orig.copy()

    # Update dmag column for {$band}_des - {$band}_ps1...
    df.loc[:,'dmag'] = df.loc[:,magName] - df.loc[:,magName_match]

    # Update mask...
    mask1 = abs(df['dmag']) <= 10.
    mask2 = ( ( df[color_name_1] > -3. ) & ( df[color_name_1] < 6.0 ) )
    mask = mask & mask1 & mask2

    # make a copy of original df, overwriting the old one...
    df = df[mask].copy()

    # Identify dmag and color1 series...
    dmag =  df.loc[:,'dmag']
    color1 = df.loc[:,color_name_1]


    # Calculate the color range to bin and the number of bins, 
    #  based on the 0.01%-99.99% color range...
    color1_desc = df[color_name_1].describe(percentiles=[0.0001, 0.9999])
    color1_min = math.floor(10*color1_desc['0.01%'])/10.
    color1_max = math.ceil(10*color1_desc['99.99%'])/10.
    if verbose>1:
        print 
        print color1_desc
        print color1_min, color1_max 
        print 
    binsize = color_binsize
    ranges = np.arange(color1_min,color1_max+binsize,binsize)
    # Label the bins by their mid-point...
    labels = np.arange(color1_min,color1_max,binsize) + 0.5*binsize
    
    #nbins = int((color1_max-color1_min)/binsize)+1
    #ranges = np.linspace(color1_min, color1_max, nbins+1)
    ## Label the bins by their mid-point...
    #labels = np.linspace(color1_min, color1_max-binsize, nbins) + 0.5*binsize

    #nbins = math.floor((color1_max-color1_min)/binsize) + 1
    #ranges = np.linspace(color1_min, color1_max, nbins)
    ## Label the bins by their mid-point...
    #labels = np.linspace(color1_min, color1_max-binsize, nbins-1) + 0.5*binsize
    
    if verbose:
        print ranges
        print labels
    
    # Bin the data, adding "bin_interval" and "bin_label' to the df dataframe...
    df['bin_interval'] = pd.cut(df[color_name_1], bins=ranges, precision=0)
    df['bin_label'] = pd.cut(df[color_name_1], bins=ranges, labels=labels)
    if verbose>1:
        print 
        print df[[color_name_1,'bin_interval','bin_label']].head()
        print 

    # Perform pandas grouping/aggregating functions on sigma-clipped Data Frame...
    df_grp = df.groupby(['bin_label'])
    bin_interval = df_grp['bin_interval'].first()
    dmag_num = df_grp['dmag'].count()
    dmag_mean = df_grp['dmag'].mean()
    dmag_sd = df_grp['dmag'].std()
    dmag_err = dmag_sd/np.sqrt(dmag_num-1)
    dmag_median = df_grp['dmag'].median()
    dmag_rsd = df_grp['dmag'].apply(robustStdDev)
    dmag_unc = np.sqrt(np.pi/2)*dmag_rsd/np.sqrt(dmag_num)

    # Rename some of the pandas series...
    dmag_num.name    = 'bin_num'
    dmag_mean.name   = 'bin_mean'
    dmag_sd.name     = 'bin_stddev'
    dmag_err.name    = 'bin_stderr'
    dmag_median.name = 'bin_median'
    dmag_rsd.name    = 'bin_rstddev'
    dmag_unc.name    = 'bin_unc'
    
    # Create new data frame containing all the relevant aggregate quantities...
    df1 = pd.concat( [bin_interval, dmag_num, \
                      dmag_mean, dmag_sd, dmag_err, \
                      dmag_median, dmag_rsd, dmag_unc, ], \
                    join='outer', axis=1 )
    
    if verbose>1:
        print 
        print df1
        print 
    
    
    # Create results output file...
    resultsFile = """%s.%s_%s.csv""" % (resultsFileBaseName, band, color_name_1)
    df1.to_csv(resultsFile)
    if verbose>1:
        print """Writing binned values to %s""" % (resultsFile)

    
    # Create name for QA plot output file...
    outputFileName = """qaPlot_%s.%s_%s_%s.png""" % (resultsFileBaseName, magName, magName_match, color_name_1)

    # Create title and labels for use in QA plots...
    title = """%s --> %s""" % (magName_match, magName)
    dmagLabel = """%s - %s""" % (magName, magName_match)

    # Create QA plots...
    status = interp1ColorQAPlots1(df, df1, color_name_1, color1_min, color1_max, 
                                     title, dmagLabel, outputFileName, verbose)
    
    




# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
V
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 


count     7074.000000
mean         1.122367
std          0.644716
min         -0.707008
0.01%       -0.703738
50%          1.018211
99.99%       3.369631
max          3.407789
Name: gi_des, dtype: float64
-0.8 3.4

[-8.00000000e-01 -7.00000000e-01 -6.00000000e-01 -5.00000000e-01
 -4.00000000e-01 -3.00000000e-01 -2.00000000e-01 -1.00000000e-01
 -2.22044605e-16  1.00000000e-01  2.00000000e-01  3.00000000e-01
  4.00000000e-01  5.00000000e-01  6.00000000e-01  7.00000000e-01
  8.00000000e-01  9.00000000e-01  1.00000000e+00  1.10000000e+00
  1.20000000e+00  1.30000000e+00  1.40000000e+00  1.50000000e+00
  1.60000000e+00  1.70000000e+00  1.80000000e+00  1.90000000e+00
  2.00000000e+00  2.10000000e+00  2.20000000e+00  2.30000000e+00
  2.40000000e+00  2.50000000e+00  2.60000000e+00  2.70000000e+00
  2.80000000e+00  2.90000000e+00  3.000

In [16]:
%ls transInterp.des_to_stetson.tingli.V_gi_des.csv


transInterp.des_to_stetson.tingli.V_gi_des.csv


In [17]:
%pwd

u'/data/des81.b/data/dtucker/TransformEqns/DES_DR2/Stetson_UBVRI'